In [12]:
#ライブラリのインポート
import cv2
import datetime

#カメラの映像取得開始（括弧内の引数はカメラの接続ID）
cap = cv2.VideoCapture(1)

#赤枠の色を指定
red = (0, 0, 255)

#オブジェクトID毎に検出時間を記録するための空の辞書を用意
object_timestamps = {}

#（要調査）フレームの平均値を保持するための変数avg
avg = None

#オブジェクトの情報を保持するための空の辞書
objects = {}  # オブジェクトの辞書 {ID: (x, y, count)}

#情報取得の無限ループ開始
while True:
    #retには映像取得が成功したかどうかが入っている（True or False)
    ret, frame = cap.read()

    #もし映像取得に失敗している場合、ループを抜ける
    if not ret:
        break
    
    #映像の処理を簡単にするため、映像をグレーに変換する
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #最初のループでは、avg変数にフレームをコピーして浮動小数点型に変換する
    #それ以降のループでは、フレームの差分を計算するためにスキップする
    if avg is None:
        avg = gray.copy().astype("float")
        continue
    
    #フレームの差分を計算するためにaccumulateWeighted関数と、
    #absdiff関数を使っている。
    #accumulateWeightedは、フレームの平均値を更新していく関数。
    #absdiffは、現在のフレームと平均フレームの差分を計算する関数。
    cv2.accumulateWeighted(gray, avg, 0.80)
    frameDelta = cv2.absdiff(gray, cv2.convertScaleAbs(avg))
    
    #画像を2値化して、輪郭を検出する
    thresh = cv2.threshold(frameDelta, 10, 255, cv2.THRESH_BINARY)[1]
    contours, hierarchy = cv2.findContours(thresh.copy(),
                                           cv2.RETR_EXTERNAL,
                                           cv2.CHAIN_APPROX_SIMPLE)
    #現在のタイムスタンプを取得する
    current_timestamp = datetime.datetime.now()
    
    #各オブジェクトのID毎に、最初のオブジェクトが検出されたタイムスタンプと、
    #最後の検出された時のタイムスタンプを記録
    #オブジェクトIDのタイムスタンプが存在しない場合には新しいエントリを作成し、
    #startとendに現在のタイムスタンプを設定する
    #すでにエントリが存在する場合は、endを現在のタイムスタンプに更新する
    for object_id, timestamps in object_timestamps.items():
        start_timestamp = timestamps["start_timestamp"]
        end_timestamp = timestamps["end_timestamp"]
        duration = end_timestamp - start_timestamp
        duration_seconds = duration.total_seconds()
        
        #動画がスタートしてから5秒以上経った時に動いたもののプリント？
        if duration_seconds >= 5:
            print(f"Object ID: {object_id}, Duration: {duration_seconds} seconds")
#         print(f"Object ID: {object_id}, Duration: {duration_seconds} seconds")
    
    for target in contours:
        x, y, w, h = cv2.boundingRect(target)
        
        if w < 100:
            continue
        
        found = False
        
        for object_id, (object_x, object_y, count) in objects.items():
            distance = abs(x - object_x) + abs(y - object_y)
            
            if distance < 200:  # 新しいオブジェクトと既存のオブジェクトが近い場合
                objects[object_id] = (x, y, count + 1)  # 既存のオブジェクトを更新
                found = True
                break
        
        if not found:  # 新しいオブジェクトが
            new_id = max(objects.keys()) + 1 if objects else 1
            objects[new_id] = (x, y, 1)  # 新しいオブジェクトを追加
            object_timestamps[new_id] = {"start_timestamp": current_timestamp, "end_timestamp": current_timestamp}
        else:  # 既存のオブジェクトが見つかった場合
            object_timestamps[object_id]["end_timestamp"] = current_timestamp
    
        cv2.rectangle(frame, (x, y), (x + w, y + h), red, 2)  # 赤い枠を描画
    
    for object_id, (object_x, object_y, count) in objects.items():
        if count > 0:
            cv2.putText(frame, f"ID: {object_id}", (object_x, object_y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, red, 2)
            cv2.putText(frame, f"Count: {count}", (object_x, object_y + 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, red, 2)
    
    cv2.imshow("camera", frame)
    # cv2.imshow("camera2", thresh)
    # cv2.imshow("camera3", frameDelta)
    
    key = cv2.waitKey(10)
    if key == 27:
        break

cap.release()
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)

2023-06-25 21:36:36.056 python3.11[1101:8780] _TIPropertyValueIsValid called with 4 on nil context!
2023-06-25 21:36:36.056 python3.11[1101:8780] imkxpc_getApplicationProperty:reply: called with incorrect property value 4, bailing.
2023-06-25 21:36:36.056 python3.11[1101:8780] Text input context does not respond to _valueForTIProperty:
2023-06-25 21:36:36.089 python3.11[1101:8780] _TIPropertyValueIsValid called with 4 on nil context!
2023-06-25 21:36:36.090 python3.11[1101:8780] imkxpc_getApplicationProperty:reply: called with incorrect property value 4, bailing.
2023-06-25 21:36:36.090 python3.11[1101:8780] Text input context does not respond to _valueForTIProperty:


Object ID: 1, Duration: 5.566279 seconds
Object ID: 4, Duration: 5.534686 seconds
Object ID: 1, Duration: 5.599625 seconds
Object ID: 3, Duration: 5.568032 seconds
Object ID: 4, Duration: 5.568032 seconds
Object ID: 1, Duration: 5.599625 seconds
Object ID: 3, Duration: 5.601283 seconds
Object ID: 4, Duration: 5.601283 seconds
Object ID: 1, Duration: 5.666672 seconds
Object ID: 3, Duration: 5.635079 seconds
Object ID: 4, Duration: 5.635079 seconds
Object ID: 1, Duration: 5.699799 seconds
Object ID: 3, Duration: 5.668206 seconds
Object ID: 4, Duration: 5.668206 seconds
Object ID: 1, Duration: 5.732517 seconds
Object ID: 3, Duration: 5.700924 seconds
Object ID: 4, Duration: 5.700924 seconds
Object ID: 1, Duration: 5.766647 seconds
Object ID: 3, Duration: 5.735054 seconds
Object ID: 4, Duration: 5.735054 seconds
Object ID: 1, Duration: 5.799229 seconds
Object ID: 3, Duration: 5.767636 seconds
Object ID: 4, Duration: 5.767636 seconds
Object ID: 6, Duration: 5.001937 seconds
Object ID: 7, Du

-1